In [4]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [5]:
history = [
    {"role": "system", "content": "Eres un asistente útil."}
]


In [6]:
def chatear():
    while True:
        pregunta = input("Tú: ")
        if pregunta.lower() in ["salir", "exit", "q"]:
            print("👋 ¡Hasta luego!")
            break

        history.append({"role": "user", "content": pregunta})

        respuesta = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=history
        )

        mensaje = respuesta.choices[0].message.content
        history.append({"role": "assistant", "content": mensaje})

        print("Asistente:", mensaje)


In [7]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import mysql.connector
from mysql.connector import Error
import os
import time

load_dotenv()

DB_TYPE = os.getenv("DB_TYPE")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

db_url = f"{DB_TYPE}+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

def lanzar_mysqlconnector():
    try:
        connection = mysql.connector.connect(
            host=DB_HOST,
            port=int(DB_PORT),
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        if connection.is_connected():
            print("🚀 Lanzamiento con mysql.connector exitoso.")
    except Error as e:
        print("❌ No se pudo lanzar con mysql.connector:", e)
    finally:
        if 'connection' in locals() and connection.is_connected():
            connection.close()
            print("🛑 Conexión mysql.connector cerrada.")

def intentar_sqlalchemy():
    try:
        engine = create_engine(db_url, pool_pre_ping=True)
        with engine.connect() as conn:
            conn.execute(text("SELECT NOW();"))
            print("✅ Conexión SQLAlchemy exitosa.")
            return engine  # <--- aquí está el cambio importante
    except Exception as e:
        print("❌ Error en SQLAlchemy:", e)
        return None

# Ejecutar flujo
engine = intentar_sqlalchemy()
if not engine:
    print("🧪 Intentando lanzar mysql.connector como disparador...")
    lanzar_mysqlconnector()
    time.sleep(2)
    print("🔁 Reintentando SQLAlchemy...")
    engine = intentar_sqlalchemy()

# engine ya queda disponible para usar


✅ Conexión SQLAlchemy exitosa.


In [8]:
def chatear_con_db(client, engine):
    history = [{"role": "system", "content": "Eres un experto en bases de datos. Devuelve solo SQL válido para responder preguntas del usuario."}]
    
    while True:
        pregunta = input("Tú: ")
        if pregunta.lower() in ["salir", "exit", "q"]:
            print("👋 ¡Hasta luego!")
            break

        history.append({"role": "user", "content": pregunta})

        respuesta = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=history
        )

        sql = respuesta.choices[0].message.content.strip()
        print("🔎 SQL generado:", sql)

        try:
            with engine.connect() as conn:
                resultado = conn.execute(text(sql))
                filas = resultado.fetchall()
                columnas = resultado.keys()
                for fila in filas:
                    print(dict(zip(columnas, fila)))
        except Exception as e:
            print("❌ Error al ejecutar SQL:", e)

        history.append({"role": "assistant", "content": sql})


In [ ]:
chatear_con_db(client, engine)

Tú:  cuantas bases de datos hay


🔎 SQL generado: SELECT COUNT(*) as total_bases_de_datos FROM information_schema.schemata;
{'total_bases_de_datos': 5}


Tú:  nombralas


🔎 SQL generado: SELECT schema_name FROM information_schema.schemata;
{'SCHEMA_NAME': 'mysql'}
{'SCHEMA_NAME': 'information_schema'}
{'SCHEMA_NAME': 'performance_schema'}
{'SCHEMA_NAME': 'sys'}
{'SCHEMA_NAME': 'testdb'}


Tú:  muestrame las tablas de testdb


🔎 SQL generado: SELECT table_name FROM information_schema.tables WHERE table_schema = 'testdb';
{'TABLE_NAME': 'clientes'}
{'TABLE_NAME': 'productos'}
{'TABLE_NAME': 'ventas'}


Tú:  muestrame clientes


🔎 SQL generado: SELECT * FROM clientes;
{'id': 1, 'nombre': 'Juan', 'email': 'juan@email.com', 'fecha_registro': datetime.date(2023, 1, 15), 'apellido': 'Pérez', 'telefono': '555-1234', 'direccion': 'Calle 123', 'ciudad': 'Madrid', 'codigo_postal': '28001', 'pais': 'España', 'limite_credito': Decimal('1000.00'), 'comentarios': 'Cliente habitual', 'tipo_cliente': 'Regular'}
{'id': 2, 'nombre': 'María', 'email': 'maria@email.com', 'fecha_registro': datetime.date(2023, 2, 20), 'apellido': 'González', 'telefono': '555-5678', 'direccion': 'Avenida Central 45', 'ciudad': 'Barcelona', 'codigo_postal': '08001', 'pais': 'España', 'limite_credito': Decimal('2500.00'), 'comentarios': 'Cliente preferente', 'tipo_cliente': 'VIP'}
{'id': 3, 'nombre': 'Carlos', 'email': 'carlos@email.com', 'fecha_registro': datetime.date(2023, 3, 10), 'apellido': 'Rodríguez', 'telefono': '555-9012', 'direccion': 'Plaza Mayor 7', 'ciudad': 'Valencia', 'codigo_postal': '46001', 'pais': 'España', 'limite_credito': Decim